# Nuclei Segmentation

Let's begin with nuclei segmentation.
We can either segment the DAPI channel and then filter out the healthy nuclei, segment only the healthy nuclei or go for the ISL1 channel that has the label for the nuclei of interest.

Let's select an image and then crop out a small piece of it to make testing easier.

In [1]:
from pathlib import Path

import numpy as np
from czifile import CziFile
from napari import Viewer
from skimage.morphology import ball, erosion, remove_small_objects

In [2]:
DATA_DIR = Path(
    "../data/NS_4_Healthy_5uM_MI132_D19.czi"
)
image_handle = CziFile(DATA_DIR)
img = np.squeeze(image_handle.asarray())[2, :, 1250:2000, 1250:2000]

In [3]:
scale = {
    values_dict["Id"]: values_dict["Value"] * 10**6
    for values_dict in image_handle.metadata(raw=False)["ImageDocument"]["Metadata"][
        "Scaling"
    ]["Items"]["Distance"]
}
spacing = (scale["Z"], scale["Y"], scale["X"])
spacing

(0.75, 0.18130982905982898, 0.18130982905982898)

## Preprocessing image

We will reduce noise in the image by applying a gaussian filter to it.

In [4]:
from skimage.filters import gaussian # noqa

In [5]:
smoothed_img = gaussian(img, sigma=(1/spacing[0], 0.5/spacing[1], 0.5/spacing[2]))

In [6]:
viewer = Viewer()
viewer.add_image(img, scale=spacing, visible=False)
viewer.add_image(smoothed_img, scale=spacing, visible=True)

<Image layer 'smoothed_img' at 0x23783015fd0>

## Cellpose

Let's begin by trying out cellpose.

In [7]:
from cellpose import models  # noqa
from cellpose.io import logger_setup  # noqa

logger_setup()

2025-02-26 14:34:16,885 [INFO] WRITING LOG OUTPUT TO C:\Users\aguco599\.cellpose\run.log
2025-02-26 14:34:16,886 [INFO] 
cellpose version: 	3.0.11 
platform:       	win32 
python version: 	3.12.7 
torch version:  	2.2.2


(<Logger cellpose.io (INFO)>,
 WindowsPath('C:/Users/aguco599/.cellpose/run.log'))

In [8]:
model = models.Cellpose(model_type="cyto3")

2025-02-26 14:34:16,894 [INFO] >>>> using CPU
2025-02-26 14:34:16,895 [INFO] >>>> using CPU
2025-02-26 14:34:16,896 [INFO] >> cyto3 << model set to be used
2025-02-26 14:34:16,930 [INFO] >>>> loading model C:\Users\aguco599\.cellpose\models\cyto3
2025-02-26 14:34:16,984 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


After a couple attempts with several algorithms, I thought there might be issues with all the saturated nuclei.
To circunvent this, I made a mask for those pixels, calculated the distance transform and added this to the original image in order to have some values here.

In [9]:
anisotropy = scale["Z"] / scale["X"]
print(anisotropy)

4.136565589902539


In [10]:
diameter = 10 / scale["X"]
print(diameter)

55.15420786536719


In [11]:
masks, flows, styles, diams = model.eval(
    [smoothed_img],
    batch_size=8,
    diameter=diameter,
    z_axis=0,
    do_3D=True,
    channels=[0, 0],
    # stitch_threshold=0.5,
    normalize=True,
    flow_threshold=0.2,
    cellprob_threshold=4.0,
    anisotropy=anisotropy,
)

2025-02-26 14:34:17,022 [INFO] channels set to [0, 0]
2025-02-26 14:34:17,023 [INFO] ~~~ FINDING MASKS ~~~
2025-02-26 14:34:17,024 [INFO] multi-stack tiff read in as having 74 planes 1 channels
2025-02-26 14:34:18,337 [INFO] running YX: 74 planes of size (750, 750)
2025-02-26 14:35:27,358 [INFO] 100%|##########| 19/19 [01:08<00:00,  3.63s/it]
2025-02-26 14:35:28,070 [INFO] running ZY: 750 planes of size (74, 750)
2025-02-26 14:38:22,178 [INFO] 100%|##########| 69/69 [02:53<00:00,  2.52s/it]
2025-02-26 14:38:23,567 [INFO] running ZX: 750 planes of size (74, 750)
2025-02-26 14:41:10,443 [INFO] 100%|##########| 69/69 [02:46<00:00,  2.42s/it]
2025-02-26 14:41:11,537 [INFO] network run in 414.16s
2025-02-26 14:41:58,720 [INFO] masks created in 47.18s
2025-02-26 14:42:01,405 [INFO] >>>> TOTAL TIME 464.38 sec


In [12]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(masks[0], scale=spacing)

<Labels layer 'Labels' at 0x237b1805100>

It's working quite well for most of the nuclei.
Some are still over segmented and this needs some filtering of small objects.

In [13]:
voxel_size = scale["X"] * scale["Y"] * scale["Z"]
print(f"voxel size is {voxel_size}")
minimum_nuclei_volume = 2000 * voxel_size
print(
    f"The minimum size for an acceptable nuclei "
    f"size would be {minimum_nuclei_volume} um3"
)

voxel size is 0.024654940585278305
The minimum size for an acceptable nuclei size would be 49.30988117055661 um3


In [14]:
from skimage.util import apply_parallel  # noqa

label = apply_parallel(erosion, masks[0], chunks=(1, masks[0].shape[1], masks[0].shape[2]), compute=True, extra_keywords={"footprint": ball(4)}, dtype="uint16")
label = remove_small_objects(label, minimum_nuclei_volume / voxel_size)

In [15]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(label, scale=spacing)

<Labels layer 'label' at 0x237b39ffe60>

After this smoothing of the labels, segmentation of nuclei looks quite fine.